In [1]:
import pandas as pd
import numpy as np
import re
from sklearn import set_config
import sklearn.metrics as metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit

In [2]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [3]:
set_config(display='diagram')

In [49]:
def canonizer(data, test_size=0.1, nb_coef=20, random_state=42, cross_validation=False, kernel='rbf'):
    
    df_results = pd.DataFrame()
    
    if cross_validation == True:
        pipe = make_pipeline(StandardScaler(), SVC(kernel=kernel, probability=True))
        cv_results = cross_validate(pipe, data.drop(['canonicity_rate'], axis=1), data['canonicity_rate'], cv=cv)
        return pipe, cv_results
    
    else:
        pipe = make_pipeline(StandardScaler(), SVC(kernel=kernel, probability=True))   

        X_train, X_test, y_train, y_test = train_test_split(data.drop(['canonicity_rate'], axis=1), data['canonicity_rate'], test_size=test_size, random_state=random_state)        

        pipe.fit(X_train, y_train)
        
        print(metrics.classification_report(y_test, pipe.predict(X_test)))
        
        # for non binary labels
        canonicity_predictions = pipe.predict(X_test)
        r2 = r2_score(y_test, canonicity_predictions)
        mae = mean_absolute_error(y_test, canonicity_predictions)
        mse = mean_squared_error(y_test, canonicity_predictions)
        rmse = mean_squared_error(y_test, canonicity_predictions, squared=False)
        print("r2 : %0.1f%% " % (r2 * 100))
        print("mean_absolute_error : ", mae)
        print("mean_squared_error : ", mse)
        print("root mean_squared_error : ", rmse)
        
        if kernel == 'linear':
            coefs = pipe.named_steps['svc'].coef_
            plot_coefficients(*coefs, data.columns, nb_coef)
            
        df_results['metadata'] = y_test
        df_results['proba canon'] = pipe.predict_proba(X_test)[:,0]
        df_results['proba non-canon'] = pipe.predict_proba(X_test)[:,1]
        df_results['prediction']= pipe.predict(X_test)
    
        df_results['accord'] = [True if row['metadata'] == row['prediction'] else False for index, row in df_results.iterrows()]
    
    
    return pipe, df_results

# BoW features 

In [4]:
df_main = pd.read_csv(r'../features/chunks_BoW_features2.csv')
df_main.set_index("index", inplace = True)
df_main = df_main.replace(np.nan, 0)

In [29]:
def retrieve_canonicity(index_name):
    list_canonicity = []
    for titre in index_name:
        temp_tag = titre.split('_')[3]
        list_canonicity.append(temp_tag)
    return list_canonicity

In [7]:
list_canonicity = retrieve_canonicity(df_main.index)

In [8]:
df_main['canonicity'] = list_canonicity

In [52]:
pipe, df_results = canonizer(df_main)

              precision    recall  f1-score   support

       canon       0.90      0.87      0.88       689
    noncanon       0.90      0.92      0.91       899

    accuracy                           0.90      1588
   macro avg       0.90      0.89      0.90      1588
weighted avg       0.90      0.90      0.90      1588



In [40]:
pipe, df_results = canonizer(df_main)

              precision    recall  f1-score   support

       canon       0.87      0.90      0.88      1772
    noncanon       0.87      0.83      0.85      1413

    accuracy                           0.87      3185
   macro avg       0.87      0.87      0.87      3185
weighted avg       0.87      0.87      0.87      3185



## Get canonicity ratings

In [24]:
df_corpus = pd.read_csv(r'../../working_progress/meta_data/df_corpus_rate_canonicity.csv')

In [25]:
def title(df_main):

    liste_titre = []
    
    for doc_name in list(df_main.index):
        
        date = doc_name.split("_")[0]
        auteur = doc_name.split("_")[1]
        titre = doc_name.split("_")[2]
        liste_titre.append(titre)
        
    return net_title(liste_titre)

In [26]:
def net_title(liste_titre):
    liste_result = []
    for titre in liste_titre:
        temp = re.sub("-", " ", titre)
        liste_result.append(temp.lower()) 
    return liste_result

In [41]:
def get_canonicity_rate(df_main, df_corpus):
    canonicity_list = []
    list_title = title(df_main)
    for titre in list_title:
        row = df_corpus.loc[df_corpus['titre'] == titre]
        if row.values.any():
            canonicity_list.append(int(row["canonicity_1"].values[0]))
        else:
            canonicity_list.append(0)
    return canonicity_list

In [42]:
canonicity_list = get_canonicity_rate(df_main, df_corpus)

In [43]:
df_rate = df_main.drop(["canonicity"], axis=1)

In [44]:
df_rate["canonicity_rate"] = canonicity_list

In [45]:
df_rate.canonicity_rate.value_counts()

0    16417
3     9542
5     5549
2      333
Name: canonicity_rate, dtype: int64

In [46]:
df_rate.canonicity_rate.value_counts(normalize=True)

0    0.515593
3    0.299677
5    0.174272
2    0.010458
Name: canonicity_rate, dtype: float64

In [50]:
pipe_rate, df_results_rate = canonizer(df_rate)

              precision    recall  f1-score   support

           0       0.82      0.91      0.86      1656
           2       1.00      0.21      0.35        33
           3       0.69      0.66      0.68       962
           5       0.75      0.59      0.66       534

    accuracy                           0.77      3185
   macro avg       0.81      0.59      0.64      3185
weighted avg       0.77      0.77      0.76      3185

r2 : 44.8% 
mean_absolute_error :  0.6621664050235478
mean_squared_error :  2.118995290423862
root mean_squared_error :  1.4556769182836766


# LDA features 

In [15]:
df_lda = pd.read_csv(r'../features/lda_features.csv')
df_lda.set_index("Unnamed: 0", inplace = True)
df_lda = df_lda.replace(np.nan, 0)

In [31]:
list_canonicity = retrieve_canonicity(df_lda.index)
df_lda['canonicity'] = list_canonicity

In [39]:
pipe_lda, df_results_lda = canonizer(df_lda)

              precision    recall  f1-score   support

       canon       0.60      0.77      0.68      1738
    noncanon       0.59      0.39      0.47      1447

    accuracy                           0.60      3185
   macro avg       0.60      0.58      0.57      3185
weighted avg       0.60      0.60      0.58      3185



In [35]:
df_lda_50 = pd.read_csv(r'../features/lda_features_50_topics.csv')
df_lda_50.set_index("Unnamed: 0", inplace = True)
df_lda_50 = df_lda.replace(np.nan, 0)

In [36]:
list_canonicity = retrieve_canonicity(df_lda_50.index)
df_lda['canonicity'] = list_canonicity

In [37]:
pipe_lda_50, df_results_lda_50 = canonizer(df_lda_50)

              precision    recall  f1-score   support

       canon       0.60      0.77      0.68      1738
    noncanon       0.59      0.39      0.47      1447

    accuracy                           0.60      3185
   macro avg       0.60      0.58      0.57      3185
weighted avg       0.60      0.60      0.58      3185



In [38]:
pipe_lda_50

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(probability=True))])

# DBoW neural embeddings features 